In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("base propre.csv", sep=",")
data

# Suppression des NA

In [ ]:
data.isna().sum()

In [ ]:
data.dropna(axis=0, subset=['Jour','Chaine','Part_de_marche','Type_prgm1','Type_prgm2'],inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data

In [ ]:
#data.to_csv("Donnees_sans_na.csv",index=False)

# Recodage par label encoder

In [ ]:
from sklearn.preprocessing import (OneHotEncoder,
                                   LabelEncoder,
                                   StandardScaler,
                                   Normalizer,
                                   MinMaxScaler,
                                  )

In [ ]:
#data['Jour'] = data['Jour'].astype('category')
#data['Jour'].cat.categories = [7,4,1,2,3,6,5] 
#data['Jour'].astype('float')
#Fonctionne mais n'utilise pas label encoder

In [ ]:
gestion_jour = LabelEncoder()
gestion_mois = LabelEncoder()
data['Jour'] = gestion_jour.fit_transform(data["Jour"].astype(str))
data['Mois'] = gestion_mois.fit_transform(data["Mois"].astype(str))

In [ ]:
gestion_heure_prgm1 = LabelEncoder()
gestion_heure_prgm2 = LabelEncoder()
data['Heure_prgm1'] = gestion_heure_prgm1.fit_transform(data["Heure_prgm1"].astype(str))
data['Heure_prgm2'] = gestion_heure_prgm2.fit_transform(data["Heure_prgm2"].astype(str))

In [ ]:
gestion_age_prgm1 = LabelEncoder()
gestion_age_prgm2 = LabelEncoder()
data['Age_conseille_prgm1'] = gestion_age_prgm1.fit_transform(data["Age_conseille_prgm1"].astype(str))
data['Age_conseille_prgm2'] = gestion_age_prgm2.fit_transform(data["Age_conseille_prgm2"].astype(str))

In [ ]:
gestion_type_prgm1 = LabelEncoder()
data['Type_prgm1'] = gestion_type_prgm1.fit_transform(data["Type_prgm1"].astype(str))

In [ ]:
reste = data[["Jour","Week end","Mois","Annee","Heure_prgm1","Duree_prgm1", "Nbre_episodes_prgm1","Age_conseille_prgm1",
              "Heure_prgm2","Duree_prgm2", "Nbre_episodes_prgm2","Age_conseille_prgm2", "Part_de_marche"]].values

# Recodage par One Out Encoder

In [ ]:
gestion_chaine = OneHotEncoder()
chaine_ohe = gestion_chaine.fit_transform(data["Chaine"]
                                  .astype(str)
                                  .values
                                  .reshape(-1, 1)
                                 )

In [ ]:
gestion_type_prgm2 = OneHotEncoder()
type_prgm2_ohe = gestion_type_prgm2.fit_transform(data["Type_prgm2"]
                                  .astype(str)
                                  .values
                                  .reshape(-1, 1)
                                 )

# Séparation variables explicatives et à expliquer

In [ ]:
X = np.concatenate((chaine_ohe.toarray(),type_prgm2_ohe.toarray(),reste), axis=1)

In [ ]:
X.shape

In [ ]:
X

In [ ]:
y = data["Type_prgm1"].values

In [ ]:
y.shape

# Séparation des échantillons

In [ ]:
from sklearn.model_selection import (train_test_split)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.9)

# Normalisation par Standard Scaler et MinMaxScaler

In [ ]:
ss = StandardScaler()
mm = MinMaxScaler()

In [ ]:
X_ess = ss.fit_transform(X_train)
X_emm = mm.fit_transform(X_train)

# Gaussian process

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
from sklearn.gaussian_process.kernels import RBF

In [ ]:
kernel = 1.0 * RBF(1.0)
gpc = GPC(kernel=kernel,random_state=0).fit(X_train, y_train)

In [ ]:
gpc.kernel

In [ ]:
gpc.score

In [ ]:
gpc.score(X_train, y_train)

# Gaussian process - standard scaler

In [ ]:
kernel = 1.0 * RBF(1.0)
gpc_ss = GPC(kernel=kernel,random_state=0).fit(X_ess, y_train)

In [ ]:
gpc_ss.score

In [ ]:
gpc_ss.score(X_ess, y_train)

# Gaussian process - MinMax scaler

In [ ]:
kernel = 1.0 * RBF(1.0)
gpc_mm = GPC(kernel=kernel,random_state=0).fit(X_emm, y_train)

In [ ]:
gpc_mm.score

In [ ]:
gpc_mm.score(X_emm, y_train)